In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = (17,13)
%matplotlib inline

In [ ]:
import numpy as np
import dill
from collections import defaultdict
from sklearn import metrics

# Hierarchical partitions as lists of lists

A hierarchical partition can be represented by nested list. However, not every construction of nested list constitutes a hierarchical partition. In a hierarchical partition a list either contain other list or either contain elements. It cannot contain both at the same time. So, for example, the following is a well formed hierarchical partition

[[1],[2,3]]

while this below other is not (pay attention to element "1")

[1,[2,3]]

because the root contains element "1" and list [2,3].

In [ ]:
def check(hp):
    """Checks if a hierarchical partition represented by nested lists is well formed or not. Namely, it ensures that each list either contain other list or elements, but not lists and elements at the same time.
    
    Examples:
    >>> hp=[[1],[2,3]]
    >>> check(hp)
    True
    >>> hp=[1,[2,3]]
    >>> check(hp)
    False
    """
    flag=None
    for chp in hp:
        if isinstance(chp,list):
            if flag is None:
                flag=True
            elif not flag:
                return False
            if not check(chp):
                return False            
        else:
            if flag is None:
                flag=False
            elif flag:
                return False
    return True

In [ ]:
hp=[[1],[2,3]]
check(hp)

In [ ]:
hp=[1,[2,3]]
check(hp)

# Flattenator

In [ ]:
def flattenator(newick):
    """Takes a hierarchical partition represented by nested lists and return a list of all its elements.
    
    Example
    >>> hp = [[3, 4, 5, 6], [[0], [1, 2]], [[7], [8, 9]]]
    >>> sorted(flattenator(hp))
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    """
    for e in newick:
        if isinstance(e,list):
            for ee in flattenator(e):
                yield ee
        else:
            yield e

In [ ]:
hp = [[3, 4, 5, 6], [[0], [1, 2]], [[7], [8, 9]]]

In [ ]:
sorted(flattenator(hp))

# Shuffling hierarchical partitions

In [ ]:
def replicate_hierarchical_partition(hp,d):
    """Replicates a hierarchical partition hp into a new one whose elements are interachanged by others as specified by the list or dictionary d."""
    if isinstance(hp,list):
        return [replicate_hierarchical_partition(chp,d) for chp in hp]
    else:
        return d[hp]

In [ ]:
d=list(range(10))
d

In [ ]:
np.random.shuffle(d)
for i in range(len(d)):
    print(i,"->",d[i])

In [ ]:
hp=[[[[5], [[[7, 6]]]]], [[2], [0]], [[[[9, 4]]], [3, 8]], [1]]
replicate_hierarchical_partition(hp,d)

# Running mean and std

In [ ]:
class RunningMeanStd:
    """To compute mean, variance, etc. online as values are obtained without the need to store them. 
    This method is also numerically stable.
    See Donald Knuth’s Art of Computer Programming, Vol 2, page 232, 3rd edition."""
    def __init__(self):
        self._M=0.
        self._S=0.
        self._n=0
    def push(self,x):
        self._n+=1
        oldM=self._M
        self._M=self._M+(x-self._M)/float(self._n)
        self._S=self._S+(x-oldM)*(x-self._M)
    def mean(self):
        if self._n>0:
            return self._M
        return None
    def variance(self):
        if self._n>1:
            return self._S/float(self._n-1)
        return None
    def std(self):
        v=self.variance()
        if v is None:
            return None
        return np.sqrt(v)
    def sem(self):
        if self._n>1:
            return self.std()/np.sqrt(self._n)
        return None
    def rel_err(self,tol_std=.05):
        assert tol_std>0.
        if self._n>1:
            return self.sem()/(abs(self.mean())+tol_std)
        return None
    def clear(self):
        self._M=0.
        self._S=0.
        self._n=0
    def __len__(self):
        return self._n
    def __repr__(self):
        return "RunningMeanStd{n="+str(len(self))+" mean="+str(self.mean())+" std="+str(self.std())+" sem="+str(self.sem())+"}"

In [ ]:
runms=RunningMeanStd()

In [ ]:
runms

In [ ]:
for i in range(10):
    x=np.random.random()
    runms.push(x)
    print(i,x,runms)

# Hierarchical Mutual Information

In [ ]:
def xlnx(x):
    """Returns x*log(x) for x > 0 or returns 0 otherwise."""
    if x <= 0.:
        return 0.
    return x*np.log(x)

In [ ]:
def HMI(Ut,Us):
    """
    Computes the hierarchical mutual information between two hierarchical partitions.
    
    Returns
    n_ts,HMI(Ut,Us) : where n_ts is the number of common elements between the hierarchical partitions Ut and Us.
    
    NOTE: We label by u,v the children of t,s respectively.
    
    Examples
    >>>"""
    if isinstance(Ut[0],list):
        if isinstance(Us[0],list):
            # Ut and Us are both internal nodes since they contain other lists.
            n_ts=0.
            H_uv=0.
            H_us=0.
            H_tv=0.
            mean_I_ts=0.0
            n_tv=defaultdict(float)            
            for Uu in Ut:
                n_us=0.
                for v,Uv in enumerate(Us):
                    n_uv,I_uv=HMI(Uu,Uv)
                    n_ts+=n_uv
                    n_tv[v]+=n_uv
                    n_us+=n_uv                    
                    H_uv+=xlnx(n_uv)
                    mean_I_ts+=n_uv*I_uv
                H_us+=xlnx(n_us)
            for _n_tv in n_tv.values():
                H_tv+=xlnx(_n_tv)
            if n_ts>0.:
                local_I_ts=np.log(n_ts)-(H_us+H_tv-H_uv)/n_ts
                mean_I_ts=mean_I_ts/n_ts
                I_ts=local_I_ts+mean_I_ts
                #print("... Ut =",Ut,"Us =",Us,"n_ts =",n_ts,"I_ts =",I_ts,"local_I_ts =",local_I_ts,"mean_I_ts =",mean_I_ts)
                return n_ts,I_ts
            else:
                #print("... Ut =",Ut,"Us =",Us,"n_ts =",0.0,"I_ts =",0.0)
                return 0.,0.
        else:
            # Ut is internal node and Us is leaf
            return len(set(flattenator(Ut))&set(Us)),0.
    else:
        if isinstance(Us,list):
            # Ut is leaf and Us internal node
            return len(set(flattenator(Us))&set(Ut)),0.          
        else:
            # Both Ut and Us are leaves
            return len(set(Ut)&set(Us)),0.

In [ ]:
# TEST
# Two random partitions (pay attention! these are non hierarchical)
p1 = [[19, 18, 5],[14, 16, 3],[7],[10, 8],[1, 17, 9, 4, 6, 15],[2, 13, 11],[12, 0]]
p2 = [[12, 9],[4, 2, 0, 7],[16],[5],[8, 3, 1, 14],[11, 6, 10],[18, 17, 19],[13, 15]]
HMI(p1,p2)

In [ ]:
# TEST
# Now with two hierarchical partitions
hp1 = [[23],[[[[[[16], [17]]]]]],[[12], [22, 13]],[5],[7],[24],[[[9], [[14, 2]]], [[[[[[27], [3]]]]]]],[20, 29, 18],[4],[26, 15],[[10], [21, 25]],[11],[[0, 28], [1], [6]],[19, 8]]
hp2 = [[[[0, 25], [24]], [6], [11, 28], [8]],[[[19], [[[[21], [4], [[[[[22, 7]]]]]]]]], [5]],[[3], [10, 23, 14]],[[27, 1, 16, 13, 18, 26, 9], [[[[15], [[[[[[12, 17]]]]]]]], [2, 20]], [29]]]
HMI(hp1,hp2)

In [ ]:
def HH(hp):
    """Returns the hierarchical entropy of a hierarchical partition.
    
    Note: this is not the most efficient implementation."""
    return HMI(hp,hp)[1]

In [ ]:
def HJH(hp1,hp2):
    """Returns the hierarchical joint entropy between two hierarchical partitions."""
    return HH(hp1)+HH(hp2)-HMI(hp1,hp2)[1]

In [ ]:
def mean_arit(x,y):
    return .5*(x+y)

In [ ]:
def mean_geom(x,y):
    return np.sqrt(x*y)

In [ ]:
def mean_max(x,y):
    return max(x,y)

In [ ]:
def mean_min(x,y):
    return min(x,y)

In [ ]:
def NHMI(hp1,hp2,generalized_mean=mean_arit):
    """Returns the normalized hierarchical mutual information.
    
    By default, it uses the arithmetic mean for normalization. However, another generalized mean can be provided if desired."""
    gm = generalized_mean(HH(hp1),HH(hp2))
    if gm > 0.:
        return HMI(hp1,hp2)[1]/gm
    return 0.

In [ ]:
def HCH(hp1,hp2):
    """Returns the hierarchical conditional entropy HCH(hp1|hp2)."""
    return HJH(hp1,hp2)-HH(hp2)

In [ ]:
def HVI(hp1,hp2):
    """Returns the hierarchical variation of information."""
    return HH(hp1)+HH(hp2)-2.0*HMI(hp1,hp2)[1]

In [ ]:
def diff_HVI(t,s,r):
    """Returns the HVI difference associated triangular inequality."""
    return HVI(t,s)+HVI(s,r)-HVI(t,r)

In [ ]:
def EHMI(hp1,hp2,min_num_shufflings=36,rel_err_tol=0.01,verbose=0):
    """Returns the expected hierarchical mutual information using the permutation model as the reference null model.
    When verbose=0, then the function works silently. For verbose=1,2,3 some information is shown as the process run. The larger the value, the more information."""
    assert min_num_shufflings>0
    runms=RunningMeanStd()    
    rel_err=2.*rel_err_tol
    d=sorted(flattenator(hp1))
    while rel_err>rel_err_tol or len(runms)<min_num_shufflings:
        np.random.shuffle(d)
        rhp1=replicate_hierarchical_partition(hp1,d)
        HMI_sample=HMI(rhp1,hp2)[1]
        runms.push(HMI(rhp1,hp2)[1])        
        if len(runms)>1:
            rel_err=runms.rel_err()  
        if verbose>0:
            if verbose==1:
                print(HMI_sample)
            elif verbose==2:
                print(HMI_sample,runms.mean(),rel_err)
            elif verbose==3:
                print(HMI_sample,runms.mean(),rel_err)                
                print(rhp1)
    return runms

In [ ]:
# TEST
# Now with two hierarchical partitions
hp1 = [[23],[[[[[[16], [17]]]]]],[[12], [22, 13]],[5],[7],[24],[[[9], [[14, 2]]], [[[[[[27], [3]]]]]]],[20, 29, 18],[4],[26, 15],[[10], [21, 25]],[11],[[0, 28], [1], [6]],[19, 8]]
hp2 = [[[[0, 25], [24]], [6], [11, 28], [8]],[[[19], [[[[21], [4], [[[[[22, 7]]]]]]]]], [5]],[[3], [10, 23, 14]],[[27, 1, 16, 13, 18, 26, 9], [[[[15], [[[[[[12, 17]]]]]]]], [2, 20]], [29]]]
HMI(hp1,hp2)

In [ ]:
EHMI(hp1,hp2,verbose=2)

In [ ]:
def AHMI(hp1,hp2,generalized_mean=mean_max):
    """Returns the adjusted hierarchical mutual information."""
    _EHMI = EHMI(hp1,hp2).mean()
    deno = generalized_mean(HH(hp1),HH(hp2))-_EHMI
    if deno>0.:
        return (HMI(hp1,hp2)[1]-_EHMI)/deno
    return 0.

In [ ]:
# TEST
# Now with two hierarchical partitions
hp1 = [[23],[[[[[[16], [17]]]]]],[[12], [22, 13]],[5],[7],[24],[[[9], [[14, 2]]], [[[[[[27], [3]]]]]]],[20, 29, 18],[4],[26, 15],[[10], [21, 25]],[11],[[0, 28], [1], [6]],[19, 8]]
hp2 = [[[[0, 25], [24]], [6], [11, 28], [8]],[[[19], [[[[21], [4], [[[[[22, 7]]]]]]]]], [5]],[[3], [10, 23, 14]],[[27, 1, 16, 13, 18, 26, 9], [[[[15], [[[[[[12, 17]]]]]]]], [2, 20]], [29]]]
AHMI(hp1,hp2)

# Generate random partitions and hierarchical partitions

In [ ]:
def random_partition(elements):
    """Generates a random partition of the set of <<elements>>
    
    Examples
    >>> random_partition(list(range(10)))
    [[4, 7, 6], [5, 1, 3, 9, 2, 8], [0]]
    """
    
    num_splitters=np.random.randint(len(elements))
    if num_splitters==0:
        return elements
    proto_partition=[-1]*num_splitters+elements
    np.random.shuffle(proto_partition)
    i=0
    while proto_partition[i]==-1:
        i+=1
    j=len(proto_partition)-1
    while proto_partition[j]==-1:
        j-=1
    rnd_partition = [[]]
    for e in proto_partition[i:j+1]:
        if e != -1:
            rnd_partition[-1].append(e)
        else:
            if len(rnd_partition[-1])>0:
                rnd_partition.append([])
    #if len(rnd_partition[-1])==0:
    #    rnd_partition.pop()
    return rnd_partition

In [ ]:
# TEST
random_partition(list(range(10)))

In [ ]:
# TEST for the generator of random partitions
for i in range(100):
    assert len(list(flattenator(random_partition(list(range(20))))))==20

In [ ]:
def random_hierarchical_partition(elements):
    """Generates a random hierarchical partition of the set of <<elements>>
    
    Examples
    >>> random_hierarchical_partition(list(range(10)))
    [[[[5], [[[7, 6]]]]], [[2], [0]], [[[[9, 4]]], [3, 8]], [1]]
    """
    partition=random_partition(elements)
    if not isinstance(partition[0],list):
        return elements
    hp=[]
    for part in partition:
        if len(part)>0:
            chp=random_hierarchical_partition(part)
            hp.append(chp)
    if len(hp)>0:
        return hp
    return elements

In [ ]:
# TEST
random_hierarchical_partition(list(range(10)))

In [ ]:
# TEST
HMI(hp1,hp2)

# Partial shuffling of hierarchical partitions

In [ ]:
def partial_shuffling_hierarchical_partition(hp,k,verbose=0):
    """It shuffle k randomly chosen elements of the hierarchical partition hp."""
    d=np.array(sorted(flattenator(hp)),dtype=np.int32)
    #if verbose>0:
    #    print(d)    
    rd=d.copy()
    np.random.shuffle(rd)
    for ii in range(k-1):
        i=rd[ii]
        j=rd[(ii+1)%k]
        #if verbose>1:
        #    print(i,j)
        d[i],d[j]=d[j],d[i]
    if verbose>0:
        print(d)
    return replicate_hierarchical_partition(hp,d)

In [ ]:
# TEST
hp = [[23],[[[[[[16], [17]]]]]],[[12], [22, 13]],[5],[7],[24],[[[9], [[14, 2]]], [[[[[[27], [3]]]]]]],[20, 29, 18],[4],[26, 15],[[10], [21, 25]],[11],[[0, 28], [1], [6]],[19, 8]]
rhp=partial_shuffling_hierarchical_partition(hp,30,verbose=2)
print(hp)
print(rhp)

In [ ]:
NHMI(hp,rhp)

In [ ]:
HMI(hp,rhp)[1]

In [ ]:
AHMI(hp,rhp)

# Compare AHMI against "exact" computation in the non-hierarchical case implemented in scikit-learn.

In [ ]:
# TEST
metrics.cluster.adjusted_mutual_info_score([0, 0, 1, 1], [0, 0, 1, 1])

In [ ]:
def partition_into_labels(p):
    d={}
    for i,c in enumerate(p):
        for e in c:
            d[e]=i
    l=np.array([0]*len(d))
    for e,i in d.items():
        l[e]=i
    return l

In [ ]:
p1=[[3, 8, 0],[2],[16, 14],[19, 18],[11],[12, 1, 6, 5, 13, 4, 17, 7, 15, 9, 10]]
p2=[[4, 13],[19],[11, 16, 18],[1, 6, 8, 7, 2],[17, 5, 9, 15, 12, 14],[0, 10, 3]]
l1=partition_into_labels(p1)
l2=partition_into_labels(p2)
l1,l2

### First test the HMI

In [ ]:
HMI(p1,p2)

In [ ]:
metrics.cluster.mutual_info_score(l1,l2)

### Secondly test the EHMI

In [ ]:
runms=EHMI(p1,p2)
runms

In [ ]:
# This way doesn't work. We must implement the way below...
#metrics.cluster.expected_mutual_information(l1,l2)

In [ ]:
# We must "extend" scikit-leearn with our implementation of an interface to metrics.cluster.expected_mutual_information(contingency, n_samples)
# https://github.com/scikit-learn/scikit-learn/blob/1495f6924/sklearn/metrics/cluster/supervised.py#L656
def sklearn_expected_mutual_info_score(labels_true, labels_pred,
                               average_method='warn'):
    """Expected Mutual Information between two clusterings.
    Expected Mutual Information (EMI) averages the Mutual
    Information (MI) score accounting for chance. 
    This metric is independent of the absolute values of the labels:
    a permutation of the class or cluster label values won't change the
    score value in any way.
    This metric is furthermore symmetric: switching ``label_true`` with
    ``label_pred`` will return the same score value. This can be useful to
    measure the agreement of two independent label assignments strategies
    on the same dataset when the real ground truth is not known.
    Read more in the :ref:`User Guide <mutual_info_score>`.
    Parameters
    ----------
    labels_true : int array, shape = [n_samples]
        A clustering of the data into disjoint subsets.
    labels_pred : array, shape = [n_samples]
        A clustering of the data into disjoint subsets.
    average_method : string, optional (default: 'warn')
        How to compute the normalizer in the denominator. Possible options
        are 'min', 'geometric', 'arithmetic', and 'max'.
        If 'warn', 'max' will be used. The default will change to
        'arithmetic' in version 0.22.
        .. versionadded:: 0.20
    Returns
    -------
    emi: float
    See also
    --------
    adjusted_mutual_info_score: Mutual Information adjusted by change
    mutual_info_score: Mutual Information (not adjusted for chance)
    Examples
    --------
    References
    ----------
    .. [1] `Vinh, Epps, and Bailey, (2010). Information Theoretic Measures for
       Clusterings Comparison: Variants, Properties, Normalization and
       Correction for Chance, JMLR
       <http://jmlr.csail.mit.edu/papers/volume11/vinh10a/vinh10a.pdf>`_
    .. [2] `Wikipedia entry for the Adjusted Mutual Information
       <https://en.wikipedia.org/wiki/Adjusted_Mutual_Information>`_
    """
    def check_clusterings(labels_true, labels_pred):
        """Check that the labels arrays are 1D and of same dimension.
        Parameters
        ----------
        labels_true : int array, shape = [n_samples]
            The true labels
        labels_pred : int array, shape = [n_samples]
            The predicted labels
        """
        labels_true = np.asarray(labels_true)
        labels_pred = np.asarray(labels_pred)

        # input checks
        if labels_true.ndim != 1:
            raise ValueError(
                "labels_true must be 1D: shape is %r" % (labels_true.shape,))
        if labels_pred.ndim != 1:
            raise ValueError(
                "labels_pred must be 1D: shape is %r" % (labels_pred.shape,))
        if labels_true.shape != labels_pred.shape:
            raise ValueError(
                "labels_true and labels_pred must have same size, got %d and %d"
                % (labels_true.shape[0], labels_pred.shape[0]))
        return labels_true, labels_pred    
    #if average_method == 'warn':
    #    warnings.warn("The behavior of AMI will change in version 0.22. "
    #                  "To match the behavior of 'v_measure_score', AMI will "
    #                  "use average_method='arithmetic' by default.",
    #                  FutureWarning)
    #    average_method = 'max'
    labels_true, labels_pred = check_clusterings(labels_true, labels_pred)
    n_samples = labels_true.shape[0]
    classes = np.unique(labels_true)
    clusters = np.unique(labels_pred)
    # Special limit cases: no clustering since the data is not split.
    # This is a perfect match hence return 1.0.
    if (classes.shape[0] == clusters.shape[0] == 1 or
            classes.shape[0] == clusters.shape[0] == 0):
        return 1.0
    contingency = metrics.cluster.contingency_matrix(labels_true, labels_pred, sparse=True)
    #contingency = contingency.astype(np.float64,**_astype_copy_false(contingency))
    # Calculate the expected value for the mutual information
    emi = metrics.cluster.expected_mutual_information(contingency, n_samples)
    return emi

In [ ]:
skl_emi=sklearn_expected_mutual_info_score(l1,l2)
skl_emi

In [ ]:
skl_emi-runms.mean()

In [ ]:
# Systematic TEST
print("The printed values should be small, say of order 0.01 or below.\n")
elements=list(range(10))
for sample in range(10):
    p1=random_partition(elements)
    p2=random_partition(elements)
    if not isinstance(p1[0],list):
        p1=[p1]
    if not isinstance(p2[0],list):
        p2=[p2]        
    print(p1)
    print(p2)
    runms=EHMI(p1,p2)
    l1=partition_into_labels(p1)
    l2=partition_into_labels(p2)
    skl_emi=sklearn_expected_mutual_info_score(l1,l2)
    print(skl_emi-runms.mean())
    print("")

# Generate all hierarchical partitions of size $n$

In [ ]:
def copy(t):
    """Returns a copy a hierarchical partition <<t>>"""
    if isinstance(t,list):
        return [copy(c) for c in t]
    else:
        return t

In [ ]:
# TEST
t=[[[[5], [1]], [0]], [[2, 8], [7, 9, 3, 4], [6]]]
t == copy(t)

In [ ]:
def prunned_copy(t,e):
    """Returns tt,aa where tt is a replica of t excluding the branch e of t. aa is the copied ancestor of e that exists in tt. In this way, if we like to replace subtree e by say subtree ee, then we can insert ee in aa to obtain a modifed copy tt of t where e is replaced by ee."""
    if isinstance(t,list):
        a = None
        b = []
        for c in t:
            if c is not e:
                cc,aa = prunned_copy(c,e)
                b.append(cc)
                if aa is not None:
                    a=aa
            else:
                a = b
        return b,a
    else:
        return t,None

In [ ]:
# TESTS
t=[[[[5], [1]], [0]], [[2, 8], [7, 9, 3, 4], [6]]]
e=t[0][0][0]
b,a = prunned_copy(t,e)
print(e)
print(b)
print(a)
ee=copy(e)
ee.append('*')
a.append(ee)
print(a)

In [ ]:
def replace(t,e,ee):
    """Assuming t, e and ee are hierarchical partitions represented by nested lists, this function replaces the branch e of t with another brach ee."""
    if e is t:
        return ee
    tt,aa = prunned_copy(t,e)
    if aa is not None:
        aa.append(ee)
        return tt
    else:
        return None

In [ ]:
# TEST
t=[[[[5], [1]], [0]], [[2, 8], [7, 9, 3, 4], [6]]]
e=t[0][0][0]#[0]
ee=['*']
#ee='*'
tt=replace(t,e,ee)
print(t)
print(e)
print(ee)
print(tt)

In [ ]:
# TEST
t=[[[[5], [1]], [0]], [[2, 8], [7, 9, 3, 4], [6]]]
e=t#[0][0][0]#[0]
ee=['*']
#ee='*'
tt=replace(t,e,ee)
print(t)
print(e)
print(ee)
print(tt)

In [ ]:
def depth_first_search(t):
    """Performs a depth first search exploration of a hierarchical partition t represented by nested lists. It is an iterator, yielding each visited node e of t."""
    if isinstance(t,list):
        yield t        
        for c in t:
            for cc in depth_first_search(c):
                yield cc

In [ ]:
# TEST
t=[[[[5], [1]], [0]], [[2, 8], [7, 9, 3, 4], [6]]]
for c in depth_first_search(t):
    print(c)

In [ ]:
# TEST
t=[[1]]
for c in depth_first_search(t):
    print(c)

In [ ]:
def generate_hierarchical_partitions(n):
    """Generates all hierarchical partitions of the n elements 0,1,2,....,n-1. The generated hierarchical partitions are yielded as nested lists."""
    if n==1:
        yield [0]
    else:
        for t in generate_hierarchical_partitions(n-1):
            #if n==4:
            #    print("# t =",t)
            for c in depth_first_search(t):
                if isinstance(c[0],list):
                    # c is non-leaf, so change with type B
                    cc=list(c)
                    cc.append([n-1])
                    #print("#.B cc =",cc)
                    yield replace(t,c,cc) # B                    
                else:
                    # c is leaf, so change with type A
                    cc=list(c)
                    cc.append(n-1)
                    #print("#.A cc =",cc)
                    yield replace(t,c,cc)
                # change with type C
                cc=[c,[n-1]]
                #print("#.C cc =",cc)
                yield replace(t,c,cc) # C                         

In [ ]:
def hp_list_to_tuples(hp):
    """Converts a hierarchical partition written in terms of lists, into a hierarchical partition written in terms of tuples. In this way, hierarchical partitions can be hashed to be included as elements of sets."""
    if isinstance(hp,list):
        return tuple([hp_list_to_tuples(c) for c in hp])
    return hp

In [ ]:
hp_list_to_tuples([[0], [[1], [2]]])

# Distance $d_n$

### Naive implementation

In [ ]:
ln2d2=0.5*np.log(2.0)
def d_n(t1,t2,n):
    """Computes the distance metric associated to the HVI given by
        d_n(T,S)=1-exp(-n(ln(2)/2)V(T,S))
    """
    return 1.0-np.exp(-n*ln2d2*HVI(t1,t2))

### Fast implementation